# Data Science Project

* Name: Author Name
* Email:


## TABLE OF CONTENTS 


- **[Introduction](#INTRODUCTION)<br>**
- **[OBTAIN](#OBTAIN)**<br>
- **[SCRUB](#SCRUB)**<br>
- **[EXPLORE](#EXPLORE)**<br>
- **[MODEL](#MODEL)**<br>
- **[iNTERPRET](#iNTERPRET)**<br>
- **[Conclusions/Recommendations](#CONCLUSIONS-&-RECOMMENDATIONS)<br>**
___

# INTRODUCTION

> Explain the point of your project and what question you are trying to answer with your modeling.



# OBTAIN

In [1]:
import pandas as pd

# for scraping app info from App Store
from itunes_app_scraper.scraper import AppStoreScraper

# for scraping app reviews from App Store
from app_store_scraper import AppStore

# for keeping track of timing
import datetime as dt
from tzlocal import get_localzone

# for building in wait times
import random
import time

In [2]:
# Set no limit to the number of max columns to display in pandas
pd.set_option('display.max_columns', None)

In [3]:
## Read in file containing app names and IDs
app_df = pd.read_csv('data/app-name-id.csv')
app_df

,app_name,ios_app_name,ios_app_id,ios_reviews,google_app_id,google_reviews
0,Headspace,headspace-meditation-sleep,493145008,"806,200",com.getsomeheadspace.android,"231,603"
1,Calm,calm,571800810,"1,300,000",com.calm.android,"408,135"
2,Stoic,stoic,1312926037,87,com.stoicroutine.stoic,377
3,Happify,happify-for-stress-worry,730601963,"3,400",com.happify.happifyinc,"2,764"
4,Shine,shine-calm-anxiety-stress,1293721589,"28,800",com.shinetext.shine,"8,221"
5,MindShift,mindshift-cbt-anxiety-relief,634684825,211,com.bstro.MindShift,"1,438"
6,MoodTools,moodtools-depression-aid,1012822112,127,com.moodtools.moodtools,"3,169"
7,Insight Timer,insight-timer-meditation-app,337472899,"307,100",com.spotlightsix.zentimerlite2,"129,194"
8,Moodfit,moodfit,1054458809,329,com.robleridge.Moodfit,504
9,Talkspace,talkspace-therapy-counseling,661829386,"20,400",com.talkspace.talkspaceapp,"3,551"


In [4]:
## Get list of app names and app IDs
app_names = list(app_df['ios_app_name'])
app_ids = list(app_df['ios_app_id'])

In [5]:
app_names

['headspace-meditation-sleep',
 'calm',
 'stoic',
 'happify-for-stress-worry',
 'shine-calm-anxiety-stress',
 'mindshift-cbt-anxiety-relief',
 'moodtools-depression-aid',
 'insight-timer-meditation-app',
 'moodfit',
 'talkspace-therapy-counseling',
 'moodmission',
 'sanvello-anxiety-depression',
 'minddoc-your-companion',
 'smiling-mind',
 'cbt-i-coach',
 'youper-self-guided-therapy',
 'innerhour-self-care-therapy',
 'breathe2relax',
 'rootd-panic-attack-relief',
 'ten-percent-happier-meditation',
 'pzizz-sleep-nap-focus',
 'noisli',
 'slumber-fall-asleep-insomnia',
 'whats-up-a-mental-health-app',
 'breethe-meditation-sleep',
 'meditopia-meditation-breathe',
 'betterme-calm-sleep-meditate',
 'mindfulness-with-petit-bambou',
 'fabulous-daily-routine-planner',
 'reflectly-journal-ai-diary',
 'relax-meditation-guided-mind']

In [6]:
## Set up App Store Scraper
scraper = AppStoreScraper()
app_store_list = list(scraper.get_multiple_app_details(app_ids))

https://itunes.apple.com/lookup?id=493145008&country=nl&entity=software
https://itunes.apple.com/lookup?id=571800810&country=nl&entity=software
https://itunes.apple.com/lookup?id=1312926037&country=nl&entity=software
https://itunes.apple.com/lookup?id=730601963&country=nl&entity=software
https://itunes.apple.com/lookup?id=1293721589&country=nl&entity=software
https://itunes.apple.com/lookup?id=634684825&country=nl&entity=software
https://itunes.apple.com/lookup?id=1012822112&country=nl&entity=software
https://itunes.apple.com/lookup?id=337472899&country=nl&entity=software
https://itunes.apple.com/lookup?id=1054458809&country=nl&entity=software
https://itunes.apple.com/lookup?id=661829386&country=nl&entity=software
https://itunes.apple.com/lookup?id=1140332763&country=nl&entity=software
https://itunes.apple.com/lookup?id=922968861&country=nl&entity=software
https://itunes.apple.com/lookup?id=1052216403&country=nl&entity=software
https://itunes.apple.com/lookup?id=560442518&country=nl&en

In [7]:
## Convert list of dicts to Pandas DataFrame and write to csv
app_info_df = pd.DataFrame(app_store_list)
app_info_df.to_csv('data/app_info.csv', index=False)
app_info_df

,screenshotUrls,ipadScreenshotUrls,appletvScreenshotUrls,artworkUrl60,artworkUrl512,artworkUrl100,artistViewUrl,features,supportedDevices,advisories,isGameCenterEnabled,kind,minimumOsVersion,trackCensoredName,languageCodesISO2A,fileSizeBytes,sellerUrl,formattedPrice,contentAdvisoryRating,averageUserRatingForCurrentVersion,userRatingCountForCurrentVersion,averageUserRating,trackViewUrl,trackContentRating,bundleId,genreIds,primaryGenreName,trackId,trackName,sellerName,isVppDeviceBasedLicensingEnabled,currentVersionReleaseDate,releaseNotes,primaryGenreId,currency,description,artistId,artistName,genres,price,releaseDate,version,wrapperType,userRatingCount
0,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,https://apps.apple.com/nl/developer/headspace-...,iosUniversal,"iPhone5s-iPhone5s,iPadAir-iPadAir,iPadAirCellu...",,False,software,13.0,Headspace: Meditation & Sleep,"EN,FR,DE,PT,ES",104807424,https://www.headspace.com/,Gratis,4+,4.73281,41094,4.73281,https://apps.apple.com/nl/app/headspace-medita...,4+,com.getsomeheadspace.headspace,"6013,6007",Health & Fitness,493145008,Headspace: Meditation & Sleep,Headspace Inc.,True,2021-06-29T22:31:11Z,Thanks for using Headspace! This update includ...,6013,EUR,Get happy. Stress less. Sleep soundly. Headspa...,384434796,Headspace Inc.,"Gezondheid en fitness,Productiviteit",0.00,2012-02-02T18:57:49Z,3.170.0,software,41094
1,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,https://apps.apple.com/nl/developer/calm-com/i...,iosUniversal,"iPhone5s-iPhone5s,iPadAir-iPadAir,iPadAirCellu...",,False,software,13.0,Calm,"EN,FR,DE,JA,KO,PT,ES",111981568,https://www.calm.com,Gratis,4+,4.59842,13935,4.59842,https://apps.apple.com/nl/app/calm/id571800810...,4+,com.calm.calmapp,"6013,6012",Health & Fitness,571800810,Calm,"Calm.com, Inc.",True,2021-07-01T14:00:15Z,Thanks for using Calm! This update contains mu...,6013,EUR,** Best of 2018 Award Winner ** — Apple\n\n** ...,571800813,Calm.com,"Gezondheid en fitness,Lifestyle",0.00,2013-02-22T20:09:23Z,5.21,software,13935
2,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,https://apps.apple.com/nl/developer/maciej-lob...,iosUniversal,"MacDesktop-MacDesktop,iPhone5s-iPhone5s,iPadAi...",,False,software,13.0,stoic.,"EN,FR,DE,IT,PL,ES,SV",72890368,http://stoicroutine.com/,Gratis,4+,4.73475,328,4.73475,https://apps.apple.com/nl/app/stoic/id13129260...,4+,com.mlobodzinski.Stoic,"6013,6012",Health & Fitness,1312926037,stoic.,Maciej Lobodzinski,True,2021-05-29T19:53:52Z,dear stoics.\n\nthis update brings a new onboa...,6013,EUR,stoic can help you live a happier and more tra...,560803468,Maciej Lobodzinski,"Gezondheid en fitness,Lifestyle",0.00,2017-11-15T12:04:05Z,2021.23,software,328
3,https://is3-ssl.mzstatic.com/image/thumb/Purpl...,,,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,https://apps.apple.com/nl/developer/happify-in...,,"iPhone5s-iPhone5s,iPadAir-iPadAir,iPadAirCellu...",,False,software,11.0,Happify: for Stress & Worry,"EN,FR,DE,IT,JA,PT,ZH,ES",135538688,http://www.happify.com,Gratis,4+,4.17391,23,4.17391,https://apps.apple.com/nl/app/happify-for-stre...,4+,com.happify.Happify,"6013,6012",Health & Fitness,730601963,Happify: for Stress & Worry,"Happify, Inc.",True,2021-06-11T11:18:34Z,"Everything's running faster and smoother, than...",6013,EUR,Happify's science

In [ ]:
## Set up loop to go through all apps
for app_name, app_id in zip(app_names, app_ids):
    
    # Get start time
    start = dt.datetime.now(tz=get_localzone())
    fmt= "%m/%d/%y - %T %p"
    
    # Print starting output for app
    print('---'*20)
    print('---'*20)    
    print(f'***** {app_name} started at {start.strftime(fmt)}')
    print()
    
    # Instantiate AppStore for app
    app_ = AppStore(country='us', app_name=app_name, app_id=app_id)
    
    # Scrape reviews posted since February 28, 2020 and limit to 10,000 reviews
    app_.review(how_many=5000,
                after=dt.datetime(2020, 1, 1),
                sleep=random.randint(20,25))
    
    reviews = app_.reviews
    
    # Add keys to store information about which app each review is for
    for rvw in reviews:
        rvw['app_name'] = app_name
        rvw['app_id'] = app_id
    
    # Print update that scraping was completed
    print(f"""Done scraping {app_name}. 
    Scraped a total of {app_.reviews_count} reviews.\n""")
    
    # Convert list of dicts to Pandas DataFrame and write to csv
    review_df = pd.DataFrame(reviews)
    review_df.to_csv('data/' + app_name + '.csv', index=False)
    
    # Get end time
    end = dt.datetime.now(tz=get_localzone())
    
    # Print ending output for app
    print(f"""Successfully wrote {app_name} reviews to csv
    at {end.strftime(fmt)}.\n""")
    print(f'Time elapsed for {app_name}: {end-start}')
    print('---'*20)
    print('---'*20)
    print('\n')
    
    # Wait 5 to 10 seconds to start scraping next app
    time.sleep(random.randint(5,10))

------------------------------------------------------------
------------------------------------------------------------
***** headspace-meditation-sleep started at 07/03/21 - 11:57:00 AM



2021-07-03 11:57:01,501 [INFO] Base - Initialised: AppStore('us', 'headspace-meditation-sleep', 493145008)
2021-07-03 11:57:01,502 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/headspace-meditation-sleep/id493145008
2021-07-03 11:57:23,548 [INFO] Base - [id:493145008] Fetched 14 reviews (14 fetched in total)
2021-07-03 11:58:07,658 [INFO] Base - [id:493145008] Fetched 30 reviews (30 fetched in total)
2021-07-03 11:58:51,755 [INFO] Base - [id:493145008] Fetched 47 reviews (47 fetched in total)
2021-07-03 11:59:35,844 [INFO] Base - [id:493145008] Fetched 65 reviews (65 fetched in total)
2021-07-03 12:00:19,945 [INFO] Base - [id:493145008] Fetched 82 reviews (82 fetched in total)
2021-07-03 12:01:04,048 [INFO] Base - [id:493145008] Fetched 96 reviews (96 fetched in total)
2021-07-03 12:01:48,373 [INFO] Base - [id:493145008] Fetched 108 reviews (108 fetched in total)
2021-07-03 12:02:32,675 [INFO] Base - [id:493145008] Fetched 130 reviews (130 fetched in total)
2

2021-07-03 12:59:05,603 [INFO] Base - [id:493145008] Fetched 1178 reviews (1178 fetched in total)
2021-07-03 12:59:50,464 [INFO] Base - [id:493145008] Fetched 1178 reviews (1178 fetched in total)
2021-07-03 13:00:35,242 [INFO] Base - [id:493145008] Fetched 1197 reviews (1197 fetched in total)
2021-07-03 13:01:19,901 [INFO] Base - [id:493145008] Fetched 1218 reviews (1218 fetched in total)
2021-07-03 13:02:04,576 [INFO] Base - [id:493145008] Fetched 1225 reviews (1225 fetched in total)
2021-07-03 13:02:49,565 [INFO] Base - [id:493145008] Fetched 1247 reviews (1247 fetched in total)
2021-07-03 13:03:34,424 [INFO] Base - [id:493145008] Fetched 1260 reviews (1260 fetched in total)
2021-07-03 13:04:19,282 [INFO] Base - [id:493145008] Fetched 1265 reviews (1265 fetched in total)
2021-07-03 13:05:04,096 [INFO] Base - [id:493145008] Fetched 1285 reviews (1285 fetched in total)
2021-07-03 13:05:48,985 [INFO] Base - [id:493145008] Fetched 1298 reviews (1298 fetched in total)
2021-07-03 13:06:33,

2021-07-03 14:01:42,119 [INFO] Base - [id:493145008] Fetched 2158 reviews (2158 fetched in total)
2021-07-03 14:02:26,865 [INFO] Base - [id:493145008] Fetched 2188 reviews (2188 fetched in total)
2021-07-03 14:03:11,781 [INFO] Base - [id:493145008] Fetched 2188 reviews (2188 fetched in total)
2021-07-03 14:03:56,440 [INFO] Base - [id:493145008] Fetched 2195 reviews (2195 fetched in total)
2021-07-03 14:04:41,242 [INFO] Base - [id:493145008] Fetched 2225 reviews (2225 fetched in total)
2021-07-03 14:05:26,036 [INFO] Base - [id:493145008] Fetched 2225 reviews (2225 fetched in total)
2021-07-03 14:06:10,830 [INFO] Base - [id:493145008] Fetched 2240 reviews (2240 fetched in total)
2021-07-03 14:06:55,847 [INFO] Base - [id:493145008] Fetched 2252 reviews (2252 fetched in total)
2021-07-03 14:07:40,701 [INFO] Base - [id:493145008] Fetched 2258 reviews (2258 fetched in total)
2021-07-03 14:08:25,562 [INFO] Base - [id:493145008] Fetched 2290 reviews (2290 fetched in total)
2021-07-03 14:09:10,

Done scraping headspace-meditation-sleep. 
    Scraped a total of 3062 reviews.

Successfully wrote headspace-meditation-sleep reviews to csv
    at 07/03/21 - 14:56:45 PM.

Time elapsed for headspace-meditation-sleep: 2:59:45.500953
------------------------------------------------------------
------------------------------------------------------------


------------------------------------------------------------
------------------------------------------------------------
***** calm started at 07/03/21 - 14:56:54 PM



...

2021-07-04 10:20:04,949 [INFO] Base - [id:1203637303] Fetched 4800 reviews (4800 fetched in total)
2021-07-04 10:20:53,905 [INFO] Base - [id:1203637303] Fetched 4830 reviews (4830 fetched in total)
2021-07-04 10:21:43,008 [INFO] Base - [id:1203637303] Fetched 4858 reviews (4858 fetched in total)
2021-07-04 10:22:31,870 [INFO] Base - [id:1203637303] Fetched 4881 reviews (4881 fetched in total)
2021-07-04 10:23:21,528 [INFO] Base - [id:1203637303] Fetched 4910 reviews (4910 fetched in total)
2021-07-04 10:24:11,996 [INFO] Base - [id:1203637303] Fetched 4938 reviews (4938 fetched in total)
2021-07-04 10:25:01,205 [INFO] Base - [id:1203637303] Fetched 4964 reviews (4964 fetched in total)
2021-07-04 10:25:49,966 [INFO] Base - [id:1203637303] Fetched 4987 reviews (4987 fetched in total)
2021-07-04 10:26:15,096 [INFO] Base - [id:1203637303] Fetched 5011 reviews (5011 fetched in total)


Done scraping fabulous-daily-routine-planner. 
    Scraped a total of 5011 reviews.

Successfully wrote fabulous-daily-routine-planner reviews to csv
    at 07/04/21 - 10:26:15 AM.

Time elapsed for fabulous-daily-routine-planner: 2:19:26.990197
------------------------------------------------------------
------------------------------------------------------------


------------------------------------------------------------
------------------------------------------------------------
***** reflectly-journal-ai-diary started at 07/04/21 - 10:26:25 AM



2021-07-04 10:26:26,104 [INFO] Base - Initialised: AppStore('us', 'reflectly-journal-ai-diary', 1241229134)
2021-07-04 10:26:26,104 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/reflectly-journal-ai-diary/id1241229134
2021-07-04 10:26:49,487 [INFO] Base - [id:1241229134] Fetched 9 reviews (9 fetched in total)
2021-07-04 10:27:36,532 [INFO] Base - [id:1241229134] Fetched 36 reviews (36 fetched in total)
2021-07-04 10:28:23,759 [INFO] Base - [id:1241229134] Fetched 55 reviews (55 fetched in total)
2021-07-04 10:29:10,740 [INFO] Base - [id:1241229134] Fetched 74 reviews (74 fetched in total)
2021-07-04 10:29:58,649 [INFO] Base - [id:1241229134] Fetched 94 reviews (94 fetched in total)
2021-07-04 10:30:45,671 [INFO] Base - [id:1241229134] Fetched 114 reviews (114 fetched in total)
2021-07-04 10:31:32,621 [INFO] Base - [id:1241229134] Fetched 135 reviews (135 fetched in total)
2021-07-04 10:32:19,611 [INFO] Base - [id:1241229134] Fetched 157 reviews (157 fetched i

2021-07-04 11:31:28,054 [INFO] Base - [id:1241229134] Fetched 1735 reviews (1735 fetched in total)
2021-07-04 11:32:14,739 [INFO] Base - [id:1241229134] Fetched 1739 reviews (1739 fetched in total)
2021-07-04 11:33:01,415 [INFO] Base - [id:1241229134] Fetched 1771 reviews (1771 fetched in total)
2021-07-04 11:33:48,092 [INFO] Base - [id:1241229134] Fetched 1771 reviews (1771 fetched in total)
2021-07-04 11:34:34,942 [INFO] Base - [id:1241229134] Fetched 1799 reviews (1799 fetched in total)
2021-07-04 11:35:21,840 [INFO] Base - [id:1241229134] Fetched 1812 reviews (1812 fetched in total)
2021-07-04 11:36:09,061 [INFO] Base - [id:1241229134] Fetched 1839 reviews (1839 fetched in total)
2021-07-04 11:36:56,097 [INFO] Base - [id:1241229134] Fetched 1865 reviews (1865 fetched in total)
2021-07-04 11:37:42,851 [INFO] Base - [id:1241229134] Fetched 1877 reviews (1877 fetched in total)
2021-07-04 11:38:29,522 [INFO] Base - [id:1241229134] Fetched 1912 reviews (1912 fetched in total)
2021-07-04

2021-07-04 12:36:47,784 [INFO] Base - [id:1241229134] Fetched 3462 reviews (3462 fetched in total)
2021-07-04 12:37:34,509 [INFO] Base - [id:1241229134] Fetched 3462 reviews (3462 fetched in total)
2021-07-04 12:38:21,458 [INFO] Base - [id:1241229134] Fetched 3462 reviews (3462 fetched in total)
2021-07-04 12:39:08,186 [INFO] Base - [id:1241229134] Fetched 3495 reviews (3495 fetched in total)
2021-07-04 12:39:56,046 [INFO] Base - [id:1241229134] Fetched 3535 reviews (3535 fetched in total)
2021-07-04 12:40:43,108 [INFO] Base - [id:1241229134] Fetched 3552 reviews (3552 fetched in total)
2021-07-04 12:41:30,014 [INFO] Base - [id:1241229134] Fetched 3552 reviews (3552 fetched in total)
2021-07-04 12:42:17,009 [INFO] Base - [id:1241229134] Fetched 3565 reviews (3565 fetched in total)
2021-07-04 12:43:03,890 [INFO] Base - [id:1241229134] Fetched 3605 reviews (3605 fetched in total)
2021-07-04 12:43:50,700 [INFO] Base - [id:1241229134] Fetched 3643 reviews (3643 fetched in total)
2021-07-04

# SCRUB

In [ ]:
app_names[11]

# EXPLORE

# MODEL

# iNTERPRET

# CONCLUSIONS & RECOMMENDATIONS

> Summarize your conclusions and bullet-point your list of recommendations, which are based on your modeling results.

# TO DO/FUTURE WORK

- 